In [1]:
import dgl
import math
import torch
import numpy as np
from os import path
from pathlib import Path
from dgl.data import DGLDataset
from ToyDGLDataset import ToyDGLDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

Using backend: pytorch


In [2]:
datasetName = 'ToyDataset01'
datasetDir = path.join('/home/andrew/GNN_Sandbox/GraphDatasets', datasetName)
dataset = ToyDGLDataset(datasetName, datasetDir)

Done loading data from cached files.


In [3]:
dataset.printProperties()

Num Graph classes: 2
Graph classes: [0, 1]
Number of graphs: 20
Number of all nodes in all graphs: 916
Number of all edges in all graphs: 48304
Dim node features: 5
Node feature keys: ['P_t', 'Eta', 'Phi', 'Mass', 'Type']
Dim edge features: 3
Edge feature keys: ['DeltaPhi', 'DeltaEta', 'RapiditySquared']


In [4]:
print(len(dataset))
graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

20
Graph(num_nodes=15, num_edges=210,
      ndata_schemes={'Type': Scheme(shape=(), dtype=torch.float64), 'Mass': Scheme(shape=(), dtype=torch.float64), 'Eta': Scheme(shape=(), dtype=torch.float64), 'Phi': Scheme(shape=(), dtype=torch.float64), 'P_t': Scheme(shape=(), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'RapiditySquared': Scheme(shape=(), dtype=torch.float64), 'DeltaEta': Scheme(shape=(), dtype=torch.float64), 'DeltaPhi': Scheme(shape=(), dtype=torch.float64)})
Label: 0
tensor([[ 0.0000e+00,  1.6273e-01,  6.9731e-02,  1.5032e+00,  2.3221e+01],
        [ 0.0000e+00,  4.4825e-01,  9.0394e+00,  1.0488e+00,  7.6783e+01],
        [ 1.0000e+00,  5.3471e-01,  6.4118e+00,  5.6615e+00,  8.9785e+01],
        [ 2.0000e+00,  6.7258e-01,  1.1639e+00,  2.3248e+00,  6.4804e+01],
        [ 2.0000e+00,  9.5304e-01, -1.0462e-01,  2.3367e+00,  7.2507e+01],
        [ 1.0000e+00,  6.2894e-01, -9.7801e+00,  1.4135e+00,  2.8199e+01],
        [ 0.0000e+00, 

In [5]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
splitIndices = dataset.get_split_indices()

train_sampler = SubsetRandomSampler(splitIndices['train'])
test_sampler = SubsetRandomSampler(splitIndices['test'])

train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=32, drop_last=False)
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=32, drop_last=False)

In [6]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

[Graph(num_nodes=544, num_edges=27700,
      ndata_schemes={'Type': Scheme(shape=(), dtype=torch.float64), 'Mass': Scheme(shape=(), dtype=torch.float64), 'Eta': Scheme(shape=(), dtype=torch.float64), 'Phi': Scheme(shape=(), dtype=torch.float64), 'P_t': Scheme(shape=(), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'RapiditySquared': Scheme(shape=(), dtype=torch.float64), 'DeltaEta': Scheme(shape=(), dtype=torch.float64), 'DeltaPhi': Scheme(shape=(), dtype=torch.float64)}), tensor([0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1])]


In [7]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())
print(labels)
# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

Number of nodes for each graph element in the batch: tensor([28, 27, 61, 64, 58, 66, 28, 65, 15, 38, 40, 54])
Number of edges for each graph element in the batch: tensor([ 756,  702, 3660, 4032, 3306, 4290,  756, 4160,  210, 1406, 1560, 2862])
tensor([0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1])
The original graphs in the minibatch:
[Graph(num_nodes=28, num_edges=756,
      ndata_schemes={'Type': Scheme(shape=(), dtype=torch.float64), 'Mass': Scheme(shape=(), dtype=torch.float64), 'Eta': Scheme(shape=(), dtype=torch.float64), 'Phi': Scheme(shape=(), dtype=torch.float64), 'P_t': Scheme(shape=(), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'RapiditySquared': Scheme(shape=(), dtype=torch.float64), 'DeltaEta': Scheme(shape=(), dtype=torch.float64), 'DeltaPhi': Scheme(shape=(), dtype=torch.float64)}), Graph(num_nodes=27, num_edges=702,
      ndata_schemes={'Type': Scheme(shape=(), dtype=torch.float64), 'Mass': Scheme(shape=(), dtype=torch.float64), 'Eta'

In [8]:
from dgl.nn import GraphConv
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [9]:
# Create the model with given dimensions
model = GCN(dataset.dim_nfeats, 16, dataset.num_graph_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(50):
    for batched_graph, labels in train_dataloader:
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)
        pred = model(batched_graph, nodeFeatVec)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #if epoch % 5 == 0:
    print(f'Epoch: {epoch}, Loss: {loss:.3f}')

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    nodeFeatVec = GetNodeFeatureVectors(batched_graph)
    pred = model(batched_graph, nodeFeatVec)
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)

Epoch: 0, Loss: 7.643
Epoch: 1, Loss: 4.851
Epoch: 2, Loss: 2.060
Epoch: 3, Loss: 1.065
Epoch: 4, Loss: 2.481
Epoch: 5, Loss: 2.555
Epoch: 6, Loss: 1.772
Epoch: 7, Loss: 0.755
Epoch: 8, Loss: 0.907
Epoch: 9, Loss: 1.482
Epoch: 10, Loss: 1.673
Epoch: 11, Loss: 1.484
Epoch: 12, Loss: 1.030
Epoch: 13, Loss: 0.622
Epoch: 14, Loss: 0.774
Epoch: 15, Loss: 1.122
Epoch: 16, Loss: 1.150
Epoch: 17, Loss: 0.872
Epoch: 18, Loss: 0.598
Epoch: 19, Loss: 0.648
Epoch: 20, Loss: 0.839
Epoch: 21, Loss: 0.908
Epoch: 22, Loss: 0.808
Epoch: 23, Loss: 0.632
Epoch: 24, Loss: 0.561
Epoch: 25, Loss: 0.659
Epoch: 26, Loss: 0.750
Epoch: 27, Loss: 0.709
Epoch: 28, Loss: 0.594
Epoch: 29, Loss: 0.548
Epoch: 30, Loss: 0.601
Epoch: 31, Loss: 0.656
Epoch: 32, Loss: 0.642
Epoch: 33, Loss: 0.578
Epoch: 34, Loss: 0.537
Epoch: 35, Loss: 0.559
Epoch: 36, Loss: 0.598
Epoch: 37, Loss: 0.593
Epoch: 38, Loss: 0.552
Epoch: 39, Loss: 0.528
Epoch: 40, Loss: 0.543
Epoch: 41, Loss: 0.565
Epoch: 42, Loss: 0.560
Epoch: 43, Loss: 0.53

In [10]:
print(graph.nodes[[0,1]])
print(graph.nodes[0])
print(graph.nodes[7])

NodeSpace(data={'Type': tensor([0., 0.], dtype=torch.float64), 'Mass': tensor([0.1627, 0.4483], dtype=torch.float64), 'Eta': tensor([0.0697, 9.0394], dtype=torch.float64), 'Phi': tensor([1.5032, 1.0488], dtype=torch.float64), 'P_t': tensor([23.2209, 76.7832], dtype=torch.float64), 'label': tensor([0, 1])})
NodeSpace(data={'Type': tensor([0.], dtype=torch.float64), 'Mass': tensor([0.1627], dtype=torch.float64), 'Eta': tensor([0.0697], dtype=torch.float64), 'Phi': tensor([1.5032], dtype=torch.float64), 'P_t': tensor([23.2209], dtype=torch.float64), 'label': tensor([0])})
NodeSpace(data={'Type': tensor([0.], dtype=torch.float64), 'Mass': tensor([0.7796], dtype=torch.float64), 'Eta': tensor([-3.6804], dtype=torch.float64), 'Phi': tensor([4.1229], dtype=torch.float64), 'P_t': tensor([40.7011], dtype=torch.float64), 'label': tensor([7])})


In [12]:
#print(graph.out_edges(7))
#print(graph.edges[(0,1)]) # src, dst
#print(graph.edges[0,1]) # src, dst
print(graph.edges[[0, 0], [1,2]]) # src, dst
print(graph.edges[0])
print(graph.edges[1])
#print(graph.edges[0]) # eid

EdgeSpace(data={'RapiditySquared': tensor([80.6612, 57.5137], dtype=torch.float64), 'DeltaEta': tensor([8.9697, 6.3421], dtype=torch.float64), 'DeltaPhi': tensor([0.4544, 4.1584], dtype=torch.float64)})
EdgeSpace(data={'RapiditySquared': tensor([80.6612], dtype=torch.float64), 'DeltaEta': tensor([8.9697], dtype=torch.float64), 'DeltaPhi': tensor([0.4544], dtype=torch.float64)})
EdgeSpace(data={'RapiditySquared': tensor([57.5137], dtype=torch.float64), 'DeltaEta': tensor([6.3421], dtype=torch.float64), 'DeltaPhi': tensor([4.1584], dtype=torch.float64)})


In [13]:
edgeList = GetNeighborNodes(graph, 0)
nodeFeat = GetNodeFeatureVectors(graph)
print(graph.nodes[0])
print(nodeFeat)
print(edgeList)


NodeSpace(data={'Type': tensor([0.], dtype=torch.float64), 'Mass': tensor([0.1627], dtype=torch.float64), 'Eta': tensor([0.0697], dtype=torch.float64), 'Phi': tensor([1.5032], dtype=torch.float64), 'P_t': tensor([23.2209], dtype=torch.float64), 'label': tensor([0])})
tensor([[ 0.0000e+00,  1.6273e-01,  6.9731e-02,  1.5032e+00,  2.3221e+01],
        [ 0.0000e+00,  4.4825e-01,  9.0394e+00,  1.0488e+00,  7.6783e+01],
        [ 1.0000e+00,  5.3471e-01,  6.4118e+00,  5.6615e+00,  8.9785e+01],
        [ 2.0000e+00,  6.7258e-01,  1.1639e+00,  2.3248e+00,  6.4804e+01],
        [ 2.0000e+00,  9.5304e-01, -1.0462e-01,  2.3367e+00,  7.2507e+01],
        [ 1.0000e+00,  6.2894e-01, -9.7801e+00,  1.4135e+00,  2.8199e+01],
        [ 0.0000e+00,  1.5741e-01,  5.7550e-02,  4.8663e-01,  5.7164e+01],
        [ 0.0000e+00,  7.7959e-01, -3.6804e+00,  4.1229e+00,  4.0701e+01],
        [ 1.0000e+00,  6.2136e-01, -5.5349e+00,  5.1368e+00,  1.2241e+01],
        [ 1.0000e+00,  6.4612e-01, -2.1258e-02,  5.5986e+

In [19]:
def ConcatNodeAndEdgeFeatures(graph, nodeFeat, nodeLabel: int):
    neighbors = GetNeighborNodes(graph, nodeLabel)
    nfeat = nodeFeat[nodeLabel]
    efeat = graph.edges[neighbors]
    print(nfeat)
    print(efeat)
    
ConcatNodeAndEdgeFeatures(graph, nodeFeat, 0)
#print(graph.edge_ids([0, 13], [13, 0]))

edgeFeatureVec = GetEdgeFeatureVectors(graph)
print(edgeFeatureVec[12])
print(edgeFeatureVec[182])
#print(len(edgeFeatureVec))
#print(edgeFeatureVec)

tensor([ 0.0000,  0.1627,  0.0697,  1.5032, 23.2209])
tensor([0.4544, 4.1584, 0.8216, 0.8336, 0.0896, 1.0165, 2.6198, 3.6336, 4.0954,
        3.1939, 1.8168, 3.8952, 4.5800, 2.3546], dtype=torch.float64)
tensor([111.1448,   9.4957,   4.5800])
tensor([111.1448,   9.4957,   4.5800])


In [ ]:
print(graph.edata['DeltaPhi'])
print(graph.dstdata['P_t'])